In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os

In [2]:
sift = cv2.xfeatures2d.SIFT_create()

In [3]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

In [4]:
uncropped_kpd_cache = {}
print("Building cache...")
uncropped_list = os.listdir("./test/uncropped/")
for i, uncropped_file_name in enumerate(uncropped_list):
    print(f"Computing {uncropped_file_name} ({i + 1} / {len(uncropped_list)})")
    uncropped = cv2.imread(f"./test/uncropped/{uncropped_file_name}")
    uncropped_keypoints, uncropped_descriptors = sift.detectAndCompute(uncropped, None)
    uncropped_kpd_cache[uncropped_file_name] = (uncropped_keypoints, uncropped_descriptors)
    del uncropped

Building cache...
Computing 0.png (1 / 100)
Computing 1.png (2 / 100)
Computing 10.png (3 / 100)
Computing 11.png (4 / 100)
Computing 12.png (5 / 100)
Computing 13.png (6 / 100)
Computing 14.png (7 / 100)
Computing 15.png (8 / 100)
Computing 16.png (9 / 100)
Computing 17.png (10 / 100)
Computing 18.png (11 / 100)
Computing 19.png (12 / 100)
Computing 2.png (13 / 100)
Computing 20.png (14 / 100)
Computing 21.png (15 / 100)
Computing 22.png (16 / 100)
Computing 23.png (17 / 100)
Computing 24.png (18 / 100)
Computing 25.png (19 / 100)
Computing 26.png (20 / 100)
Computing 27.png (21 / 100)
Computing 28.png (22 / 100)
Computing 29.png (23 / 100)
Computing 3.png (24 / 100)
Computing 30.png (25 / 100)
Computing 31.png (26 / 100)
Computing 32.png (27 / 100)
Computing 33.png (28 / 100)
Computing 34.png (29 / 100)
Computing 35.png (30 / 100)
Computing 36.png (31 / 100)
Computing 37.png (32 / 100)
Computing 38.png (33 / 100)
Computing 39.png (34 / 100)
Computing 4.png (35 / 100)
Computing 40.png

In [10]:
uncropped_kpd_cache['99.png'][1]

array([[  6.,   0.,   0., ...,   0.,   0.,   2.],
       [ 37.,  19.,   3., ...,   0.,   0.,   0.],
       [  0.,  11., 133., ...,   0.,   0.,   0.],
       ...,
       [ 28.,  40.,  20., ...,   0.,   0.,   1.],
       [  0.,   0.,   0., ...,   1.,   3.,   5.],
       [  0.,   1.,  99., ...,   0.,   0.,   0.]], dtype=float32)

In [5]:
def get_uncropped_from_cropped(cropped):
    cropped_keypoints, cropped_descriptors = sift.detectAndCompute(cropped, None)
    closest_match = (None, None, None)
    for uncropped_file_name in os.listdir("./test/uncropped/"):
        #print(f"- checking {uncropped_file_name}")
        uncropped = cv2.imread(f"./test/uncropped/{uncropped_file_name}")
        uncropped_keypoints, uncropped_descriptors = uncropped_kpd_cache[uncropped_file_name] #sift.detectAndCompute(uncropped, None)
        matches = flann.knnMatch(cropped_descriptors, uncropped_descriptors, k=2)
        good = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good.append(m)
        #print(f"- has {len(good)} good matches")
        if len(good) < 4:
            continue
        uncropped_points = np.float32([uncropped_keypoints[m.trainIdx].pt for m in good])
        cropped_points = np.float32([cropped_keypoints[m.queryIdx].pt for m in good])
        M = cv2.findHomography(cropped_points, uncropped_points, cv2.RANSAC, 5.0)[0]
        if M is not None:
            dst = cv2.warpPerspective(uncropped, M, (cropped.shape[1], cropped.shape[0]), flags=cv2.WARP_INVERSE_MAP)
            diff = np.average(dst.astype(float) - cropped)
            diff = abs(diff)
            #print(f"- diff: {diff}")
            if (not closest_match[0]) or closest_match[2] > diff:
                closest_match = (uncropped_file_name, M, diff)
                if diff == 0.0:
                    break
    return closest_match

In [6]:
for cropped in os.listdir("./test/cropped/"):
    cropped_img = cv2.imread(f"./test/cropped/{cropped}")
    print(f"Testing {cropped}")
    (uncropped_file_name, M, diff) = get_uncropped_from_cropped(cropped_img)
    print(f"Tested {cropped}. Got {uncropped_file_name}. M = {M}. Avg diff = {diff}")

Testing 0.png
Tested 0.png. Got 0.png. M = [[1.00013340e+00 3.41072388e-05 3.53599961e+03]
 [6.03075892e-05 1.00002114e+00 1.59599948e+03]
 [3.55987168e-08 9.03405709e-09 1.00000000e+00]]. Avg diff = 0.0
Testing 1.png
Tested 1.png. Got 1.png. M = [[ 9.99903382e-01  7.56534668e-05  5.14000240e+02]
 [-7.15325378e-05  1.00007568e+00  2.96998814e+02]
 [-1.42689071e-07  1.45286184e-07  1.00000000e+00]]. Avg diff = 0.0
Testing 10.png
Tested 10.png. Got 10.png. M = [[ 9.98009360e-01 -1.38242071e-04  1.29801605e+03]
 [-2.47682007e-04  9.99899446e-01  2.50151179e+01]
 [-1.34033498e-06  1.21287553e-07  1.00000000e+00]]. Avg diff = 0.0
Testing 11.png


KeyboardInterrupt: 